# Demo Tensorflow and Cloud Machine Learning Engine using the Iris Dataset

In this demo we will be going through the steps of building a Machine Learning Pipeline using Tensorflow and Cloud Machine Learning Engine. This demo uses and explains best practices that can be used to build a ML pipeline using GCP and Tensorflow. The end product is a ML pipeline that creates an API that can be used for online inference. 

This notebook is intended to be run on Google Cloud [Datalab](https://cloud.google.com/datalab/docs/quickstarts)

Datalab will have the required libraries installed by default for this code to work. If you choose to run this code outside of Datalab you may run in to version and dependency issues which you will need to resolve.

#### Import libraries

In [21]:
from __future__ import division
from __future__ import print_function

import argparse
import multiprocessing
import time

import tensorflow as tf
import tensorboard
from tensorflow.contrib.learn.python.learn.utils import (saved_model_export_utils)

tf.logging.set_verbosity(tf.logging.INFO)

## Tensorflow APIs

Tensorflow is a hierarchical framework. The further down the hierarchy you go, the more flexibility you have, but that more code you have to write. A best practice is start at the highest level of abstraction. Datasets and Estimators are two key TensorFlow higher level API's that are used a lot for ML projects:

- Datasets: Best practice way of creating input pipelines. 
- Estimators: A high-level API to create TensorFlow models. Estimators include canned models (out of the box) and custom estimators.

Below you the TensorFlow architecture including the dataset API an Estimators. Combined, they offer an easy way to create TensorFlow models.

![title](https://3.bp.blogspot.com/-l2UT45WGdyw/Wbe7au1nfwI/AAAAAAAAD1I/GeQcQUUWezIiaFFRCiMILlX2EYdG49C0wCLcBGAs/s1600/image6.png)

For this tutorial we will be operating at the highest level of Tensorflow abstraction, using the Estimator API.

Steps of this demo:
- 1: Get raw data
- 2: Write Tensorflow Code
    - Define Meta data and Feature Columns
    - Define Input Function and Eval Function
    - Define Serving Function
    - Initialize Estimator
    - Define Train and Eval Function
- 3: Package Code
- 4: Train on Google Cloud using Cloud Machine Learning Engine
- 5: Inspect Results using Tensorboard
- 6: Deploy Model using Cloug Machine Learning Engine
- 7: Get Predictions using the API

### 1) Get raw data (csv) 
For this demo we are using the [Iris dataset](https://archive.ics.uci.edu/ml/datasets/iris). Which is a famous Machine Learning dataset used for benchmarking and demo's. Be aware that it's a really small dataset and it's only used here to demonstrate how to build a ML pipeline using Tensorflow and Cloud Machine Learning Engine. The trained model categorizes Iris flowers based on four botanical features (sepal length, sepal width, petal length, and petal width). So, during inference, you can provide values for those four features and the model will predict that the flower is one of the following three beautiful variants:

![title](https://www.tensorflow.org/images/iris_three_species.jpg)


The CSV files are available on GCS and can be downloaded here: [Iris Train](https://storage.googleapis.com/erwinh-ml-demo/models/trainer_estimator_iris/data/iris_training.csv) & [Iris Test](https://storage.googleapis.com/erwinh-ml-demo/models/trainer_estimator_iris/data/iris_test.csv).

In [5]:
# Path to training and test data in bucket
train_path = 'gs://erwinh-ml-demo/models/trainer_estimator_iris/data/iris_training.csv'
test_path = 'gs://erwinh-ml-demo/models/trainer_estimator_iris/data/iris_test.csv'

### 2) Write Tensorflow Code

#### 2A) Setup Meta Data and Feature Columns
Feature columns are your Estimator's data "interface." They tell the estimator in what format they should expect data and how to interpret it (is it one-hot? sparse? dense? continous?). You can read more in the Tensorflow documentation on [Tensorflow Feature Columns](https://www.tensorflow.org/api_docs/python/tf/feature_column)

In [6]:
# Here we define the feature names + label. In this case we use all features. 
FEATURE_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth']
LABEL = ['Species']

In [7]:
# Set the data format that can be expected
feature_columns = [tf.feature_column.numeric_column(k) for k in FEATURE_NAMES]

#### 2B) Define Input and Evaluation Function
Now that you have an estimator and it knows what type of data to expect and how to interpret, you need to actually pass the data to it! This is the job of the input function.

The input function returns a (features, label) tuple

features: A python dictionary. Each key is a feature column name and its value is the tensor containing the data for that Feature
label: A Tensor containing the label column

In this case we have a separate function that will take care of decoding the CSV file and reading in the data that is used by the Input Function. On the Tensorflow [Tensorflow Feature Columns](https://www.tensorflow.org/programmers_guide/datasets) you can read more information on some of the best practices.

In [8]:
# This function takes care of decoding the csv file

def decode_csv(line):
    parsed_line = tf.decode_csv(line, [[0.], [0.], [0.], [0.], [0]]) # parsed line + defaults
    label = parsed_line[-1]  # Last element is the label
    del parsed_line[-1]  # Delete label
    features = parsed_line  # Everything but last element are features
    d = dict(zip(FEATURE_NAMES, features)), label 
    
    return d 

Now we will create our input function that will use the tf.data api. The [tf.data.TextLineDataset](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) will create a dataset comprising lines from a textfile (csv in this case). 

In [9]:
# Here we define our input function

def generate_input_fn(data_set):
    def input_fn(repeat_count=100, shuffle_count=500):
       dataset = (tf.data.TextLineDataset(data_set)  # Read text file
         .skip(1)  # Skip header row
         .map(decode_csv)  # uses decode_csv function
         .shuffle(shuffle_count) 
         .repeat(repeat_count) # repeat dataset count times
         .batch(30)
         .prefetch(90)  # Dataset that prefetches elements from this dataset
       )
       iterator = dataset.make_one_shot_iterator()
       batch_features, batch_labels = iterator.get_next()
       return batch_features, batch_labels # Return tuple with features and labels
    return input_fn

In [10]:
# this is the input function that we use for evaluation

def eval_input_fn(data_set):
    def eval_input_fn(repeat_count=1, shuffle_count=1):
       dataset = (tf.data.TextLineDataset(data_set)  # Read text file
         .skip(1)  # Skip header row
         .map(decode_csv, num_parallel_calls=2)  # Decode each line 
         .shuffle(shuffle_count)  
         .repeat(repeat_count)    
         .batch(30)
         .prefetch(90)  # Maximum number of elements that will be buffered when prefetching
       )
       iterator = dataset.make_one_shot_iterator()
       batch_features, batch_labels = iterator.get_next()
       return batch_features, batch_labels
    return eval_input_fn

#### 2C) Create input function for serving 
We need to define a serving input function which will be used to read inputs at prediction time.

We create this input function since we may be receiving data in a different format during serving. The serving input function performs transformation necessary to get the data provided at prediction time into the format compatible with the Estimator API.

In [11]:
# This is the input_fn we use for serving

def serving_input_fn ():
  #feature_placeholders are what the caller of the predict() method will have to provide
  feature_placeholders = {
      column.name: tf.placeholder(column.dtype, [None])
      for column in feature_columns
  }
  
  #features are what we actually pass to the estimator
  features = {
    # Inputs are rank 1 so that we can provide scalars to the server
    # but Estimator expects rank 2, so we expand dimension
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }
  return tf.estimator.export.ServingInputReceiver(
    features, feature_placeholders
  )

#### 2D) Initialize Estimator: DNN Classifier

In this example we use the [Estimator API](https://www.tensorflow.org/programmers_guide/estimators). An Estimator is what implements the training, eval and prediction loops. The Estimator uses these methods:

- fit(): training
- eval(): evaluation
- predict(): prediction
- export_savedmodel(): writing model state to disk

Tensorflow has several canned estimator that already implement these methods (DNNClassifier, LogisticClassifier etc..) or you can implement a [custom Estimator](https://www.tensorflow.org/get_started/custom_estimators). For simplicity we will use a canned [Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier).

Notice we wrap the estimator with a function. This is to allow us to specify the 'output_dir' at runtime, instead of having to hardcode it here.

In [12]:
# Here we specify some of the meta data for the model. We also have a temp directory

number_hidden_units =[256, 128] # Hidden layers + neuron's
number_classes = 3 # Number of classes
temp_direct = "output_dir/iris_%s" % (int(time.time())) # Temp directory for training locally

In [13]:
# This function wil initialize the DNN Classifier

def generate_estimator(output_dir):
  return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                    hidden_units=number_hidden_units,
                                    model_dir=output_dir,
                                    n_classes=number_classes)

#### 2E) Define Train and Eval Function
For training and evaluation we are using [tf.estimator.train_and_evaluate()](https://www.tensorflow.org/api_docs/python/tf/estimator/train_and_evaluate)

This function is important because it provides consistent behavior across local and distributed environments. The function will take care of parallelizing the computation graph across these devices (GPU & CPU)!

The tran_and_evaluate() function takes three arguments:

- estimator: mentioned earlier
- train_spec: specifies the training input function
- eval_spec: specifies the eval input function, and also the 'exporter' which uses our serving_input_fn for serving the model

In [14]:
train_spec = tf.estimator.TrainSpec(
                input_fn=generate_input_fn(train_path),
                max_steps=1000)

exporter = tf.estimator.LatestExporter('Servo', serving_input_fn)

eval_spec=tf.estimator.EvalSpec(
            input_fn=eval_input_fn(test_path),
            steps=10,
            exporters=exporter)

#### Test Run DNN Classifier
Now its time to take our classifier for a spin and train it locally. It's a best practice to first run locally on a small dataset to check for errors. First do some housekeeping and delete previous checkpoints.

In [15]:
%%bash

rm -rf output_dir/*

In [16]:
# This will train the estimator in the local environment. Accuracy can be found at the end. 

tf.estimator.train_and_evaluate(generate_estimator(temp_direct), train_spec, eval_spec)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe3f9c6fbd0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'output_dir/iris_1528992112', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


### 3) Package Code

First we need to do some housekeeping again. 

In [17]:
%%bash
rm -rf trainer

We now have all the Tensorflow code needed and it runs! To scale the training and inference using Cloud Machine Learning Engine we need to package the code into some python files and make some changes:

- Add some boilerplate code to parse the command line arguments required for gcloud.
- Use the learn_runner.run() function to run the experiment
- We also add an empty __init__.py file to the folder. This is just the python convention for identifying modules.

There is a great [blogpost](https://towardsdatascience.com/how-to-train-machine-learning-models-in-the-cloud-using-cloud-ml-engine-3f0d935294b3) with more details on how to package Python code.

In [18]:
%%bash
mkdir trainer
touch trainer/__init__.py 

In [19]:
%%writefile trainer/task.py

# Import libraries
from __future__ import division
from __future__ import print_function

import argparse
import multiprocessing
import os
import time

import tensorflow as tf
import tensorboard
from tensorflow.contrib.learn.python.learn.utils import (saved_model_export_utils)

tf.logging.set_verbosity(tf.logging.INFO)

# Path to training and test data in bucket
train_path = 'gs://erwinh-ml-demo/models/trainer_estimator_iris/data/iris_training.csv'
test_path = 'gs://erwinh-ml-demo/models/trainer_estimator_iris/data/iris_test.csv'

# Specify meta data
FEATURE_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth']
LABEL = ['Species']

feature_columns = [tf.feature_column.numeric_column(k) for k in FEATURE_NAMES]

# Model hyperparameter + number of classes
number_hidden_units =[512, 256, 128] 
number_classes = 3

# Generate DNN Classifier
def generate_estimator(output_dir):
  return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                    hidden_units=number_hidden_units,
                                    model_dir=output_dir,
                                    n_classes=number_classes)

# Function to decode csv: features & labels
def decode_csv(line):
    parsed_line = tf.decode_csv(line, [[0.], [0.], [0.], [0.], [0]])
    label = parsed_line[-1]  # Last element is the label
    del parsed_line[-1]  # Delete last element
    features = parsed_line  # Everything but last elements are the features
    d = dict(zip(FEATURE_NAMES, features)), label
    
    return d

# Input function that uses the decode csv to return features and labels
def generate_input_fn(data_set):
    def input_fn(repeat_count=100, shuffle_count=500):
       dataset = (tf.data.TextLineDataset(data_set)  # Read text file
         .skip(1)  # Skip header row
         .map(decode_csv)  # Decode each line 
         .shuffle(shuffle_count)  # Randomize elems (1 == no operation)
         .repeat(repeat_count)    # Repeats dataset this # times
         .batch(32)
         .prefetch(150)  # Maximum number of elements that will be buffered when prefetching
       )
       iterator = dataset.make_one_shot_iterator()
       batch_features, batch_labels = iterator.get_next()
       return batch_features, batch_labels
    return input_fn

# Input function used for evaulation data
def eval_input_fn(data_set):
    def eval_input_fn(repeat_count=1, shuffle_count=1):
       dataset = (tf.data.TextLineDataset(data_set)  # Read text file
         .skip(1)  # Skip header row
         .map(decode_csv, num_parallel_calls=2)  # Decode each line 
         .shuffle(shuffle_count)  # Randomize elems (1 == no operation)
         .repeat(repeat_count)    # Repeats dataset this # times
         .batch(32)
         .prefetch(150)  # Maximum number of elements that will be buffered when prefetching
       )
       iterator = dataset.make_one_shot_iterator()
       batch_features, batch_labels = iterator.get_next()
       return batch_features, batch_labels
    return eval_input_fn

# Serving input function that will be used for online inference
def  serving_input_fn ():
  #feature_placeholders are what the caller of the predict() method will have to provide
  feature_placeholders = {
      column.name: tf.placeholder(column.dtype, [None])
      for column in feature_columns
  }
  
  #features are what we actually pass to the estimator
  features = {
    # Inputs are rank 1 so that we can provide scalars to the server
    # but Estimator expects rank 2, so we expand dimension
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }
  return tf.estimator.export.ServingInputReceiver(
    features, feature_placeholders
  )

# Setup Train Spec, Evaluation Spec and Exporter.
train_spec = tf.estimator.TrainSpec(
                input_fn=generate_input_fn(train_path),
                max_steps=2000)

exporter = tf.estimator.LatestExporter('Servo', serving_input_fn)

eval_spec=tf.estimator.EvalSpec(
            input_fn=eval_input_fn(test_path),
            steps=10,
            exporters=exporter)

######START CLOUD ML ENGINE BOILERPLATE######
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  parser.add_argument(
        '--job-dir',
        help='this model ignores this field, but it is required by gcloud',
        default='junk'
    )
  args = parser.parse_args()
  arguments = args.__dict__
  output_dir = arguments.pop('output_dir')
  
  ######END CLOUD ML ENGINE BOILERPLATE######

  #initiate training job
  tf.estimator.train_and_evaluate(generate_estimator(output_dir), train_spec, eval_spec)

Writing trainer/task.py


### 4) Train on Google Cloud using Cloud Machine Learning Engine

Now that our code is packaged we can invoke it using the gcloud command line tool to run a job using Cloud Machine Learning Engine.

Note: Since our dataset is so small and our model is simple the overhead of provisioning the cluster is longer than the actual training time. Accordingly you'll notice the single VM cloud training takes longer than the local training, and the distributed cloud training takes longer than single VM cloud. For larger datasets and more complex models this will reverse

We'll need to create environment variables for our project name and Google Cloud Storage Bucket. Keep your data (bucket) and Cloud Machine Learning Engine environment in the same region. 

You need to change some of the variables below:

In [20]:
# Change these variables to you own

GCS_BUCKET = 'gs://erwinh-ml-demo/models/output_iris' # CHANGE THIS TO YOUR BUCKET 
PROJECT = 'erwinh-mldemo' # CHANGE THIS TO YOUR PROJECT ID
REGION = 'us-central1' # OPTIONALLY CHANGE THIS

In [22]:
import os
os.environ['GCS_BUCKET'] = GCS_BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [23]:
%%bash
gcloud config set project $PROJECT

Updated property [core/project].


#### 4A) Train Local
You can also use the gcloud command to train your model locally. 

In [ ]:
%%bash
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=trainer \
   -- \
   --output_dir='./output'

#### 4B) Train in the Cloud on one ML unit: n1-standard-4 machine
Here we are training in the cloud. We add a unique jobnime using time and date. Then we go and submit the job to Cloud Machine Learning Engine using the gcloud ml-eninge command. 

In [24]:
%%bash 

JOBNAME=iris_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME/ \
   --runtime-version 1.8 \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output

jobId: iris_180614_162024
state: QUEUED


Job [iris_180614_162024] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe iris_180614_162024

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs iris_180614_162024


#### 4C) Train in the Cloud on STANDARD_1

This will consist of: master: n1-highcpu-8, workers: n1-highcpu-8, parameter servers: n1-standard-4. 

Because we are using the TF Estimators interface distributed training is easy. We are changing the scale-tier that we want to use. In the [documentation](https://cloud.google.com/ml-engine/pricing#ml_training_units_by_scale_tier) you can find more information on the different tier options. 



In [25]:
%%bash 

JOBNAME=iris_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME \
   --runtime-version 1.8 \
   --scale-tier=STANDARD_1 \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output

jobId: iris_180614_162137
state: QUEUED


Job [iris_180614_162137] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe iris_180614_162137

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs iris_180614_162137


#### 4D) Train on GPU's
The "BASIC_GPU" corresponds to one Tesla K80 at the time of this writing, hardware subject to change. Bew aware that it takes some time to spin up the recources. 

In [86]:
%%bash 

JOBNAME=iris_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME \
   --runtime-version 1.8 \
   --scale-tier=BASIC_GPU \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output

jobId: iris_180611_164636
state: QUEUED


Job [iris_180611_164636] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe iris_180611_164636

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs iris_180611_164636


#### 4F) Run on custom Tier 
It's possible to create a custom tier. In the [documentation](https://cloud.google.com/ml-engine/docs/pricing) you can read about the machines you can select and the pricing. First we create a yaml file to specify the:

- Tier type
- Master and worker type
- WorkerCount

I choose for the comples_model_s type in the case

In [91]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: complex_model_s
  workerType: complex_model_s
  workerCount: 1

Overwriting config.yaml


After creating the config file we can add it to the gcloud command and run the job. This will take a bit more time to run since it needs to spin up the specific machines. 

In [92]:
%%bash 

JOBNAME=iris_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME \
   --runtime-version 1.8 \
   --config config.yaml \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output

jobId: iris_180611_165912
state: QUEUED


Job [iris_180611_165912] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe iris_180611_165912

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs iris_180611_165912


### 5) Inspect results using Tensorboard

Tensorboard is a utility that allows you to visualize your results. To make it work you need to change the path to where you have written your output. 

In [27]:
%%bash
cd $GCS_BUCKET/$JOBNAME/output

bash: line 1: cd: gs://erwinh-ml-demo/models/output_iris//output: No such file or directory


In [28]:
from google.datalab.ml import TensorBoard
TensorBoard().start('output') 

TensorBoard was started successfully with pid 9919. Click here to access it.

9919

This code will stop Tensorboard

In [29]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)

### 6) Deploy the model as an API using Cloud Machine Learning Engine

Here you need to change and check some of the variables below. Please check:

- MODEL_NAME: Give this a name (you can leave it like this). Make sure it unique.  
- MODEL_VERSION: Specify a version. 
- MODEL_LOCATION: Change this to where your model is saved. You can find it in the output/export/servo directory. 

In [30]:
%%bash
MODEL_NAME="Iris_Model"
MODEL_VERSION="v2"
MODEL_LOCATION='gs://erwinh-ml-demo/models/output_iris/iris_180611_103147/output/export/Servo/1528713201/' 

#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME} #Uncomment to overwrite existing version
#gcloud ml-engine models delete ${MODEL_NAME} #Uncomment to overwrite existing model
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --staging-bucket='gs://huize501'

Created ml engine model [projects/erwinh-mldemo/models/Iris_Model].
Creating version (this might take a few minutes)......
.................................................................................................done.


### 7) Get predictions

Cloud Machine Learning Engine supports two ways of doing predictions: Online (API) and Batch. 

When to use what:
- Online prediction is more appropriate for latency sensitive requests as results are returned quickly and synchronously.
- Batch prediction is more appropriate for large prediction requests that you only need to run a few times a day.

The prediction services expects prediction requests in standard JSON format so first we will create a JSON file with a couple of housing records. So we will create a JSON file that we can use to get predictions.

In [31]:
%%writefile records.json
{"SepalLength": 6.9,"SepalWidth": 3.1,"PetalLength": 5.4,"PetalWidth": 2.1}

Overwriting records.json


Now we can pass the JSON file to the API using gcloud. The results are returned immediatley.



In [32]:
!gcloud ml-engine predict --model Iris_Model --json-instances records.json

CLASS_IDS  CLASSES  LOGITS                                                         PROBABILITIES
[2]        [u'2']   [-20.320819854736328, -13.325811386108398, 33.51441192626953]  [4.16539226112411e-24, 4.5451547097935835e-21, 1.0]


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



### Recap

What Have we seen in this Notebook?

- 1: Get raw data
- 2: Write Tensorflow Code
- 3: Package Code
- 4: Train on Google Cloud using Cloud Machine Learning Engine
- 5: Inspect Results using Tensorboard
- 6: Deploy Model using Cloug Machine Learning Engine
- 7: Get Predictions using the API

What we will safe for the next demo:
- Feature engineering using Dataflow 
- Take data from BigQuery
- Hyperparameter tuning
- Training model using TPU's

This notebook is build to give you an introduction in to building Tensorflow models using Datasets and Estimators that can be trained and deployed using Cloud Machine Learning Engine. If you want to learn more it's best to have a look at our Online How Google Does Machine Learning [Course](https://www.coursera.org/learn/google-machine-learning) available on Coursera. 



